In [5]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

df = pd.read_stata('Returns/NBA/NBAFavReturnsnovig.dta')
df['AwayWins'] = 0
df['AwaySpread'] = 0
df['AwayReturns'] = 0
df['HomeWins'] = 0
df['HomeSpread'] = 0
df['HomeReturns'] = 0


teams = pd.concat([df['awayteam'],df['hometeam']]).unique()
teamDict = {}
for team in teams:
    teamDict[team] = {'wins':[], 'spread':[], 'returns':[], 'games':0}

df['year'] = df['date'] //10000
#print(df['awayspreadopenodds'])

In [6]:
year = 2006
momentum = ['wins', 'spread', 'returns']
GAMES = 1230
N = 4

In [7]:
for index, row in df.iterrows():
    awayteam = row['awayteam']
    hometeam = row['hometeam']

    curyear = row['year']
    if curyear != year:
        year = curyear
        for team in teamDict: 
            for x in momentum:
                for i in range(len(teamDict[team][x])):
                    teamDict[team][x].pop(0)
            teamDict[team]['games'] = 0

    wins = 0
    spread = 0
    returns = 0
    for i in range(len(teamDict[awayteam]['wins'])):
        wins += teamDict[awayteam]['wins'][i]
        spread += teamDict[awayteam]['spread'][i]
        returns += teamDict[awayteam]['returns'][i]
    df.loc[[index],['AwayWins']] = (wins / N)
    df.loc[[index],['AwaySpread']] = spread
    df.loc[[index],['AwayReturns']] = (returns / N)
    df.loc[[index],['AwayGames']] = teamDict[awayteam]['games']

    wins = 0
    spread = 0
    returns = 0
    for i in range(len(teamDict[hometeam]['wins'])):
        wins += teamDict[hometeam]['wins'][i]
        spread += teamDict[hometeam]['spread'][i]
        returns += teamDict[hometeam]['returns'][i]
    df.loc[[index],['HomeWins']] = (wins / N)
    df.loc[[index],['HomeSpread']] = spread
    df.loc[[index],['HomeReturns']] = (returns / N)
    df.loc[[index],['HomeGames']] = teamDict[hometeam]['games']

    if len(teamDict[awayteam]['wins']) > N-1:
        teamDict[awayteam]['wins'].pop(0)
        teamDict[awayteam]['spread'].pop(0)
        teamDict[awayteam]['returns'].pop(0)
    if len(teamDict[hometeam]['wins']) > N-1:
        teamDict[hometeam]['wins'].pop(0)
        teamDict[hometeam]['spread'].pop(0)
        teamDict[hometeam]['returns'].pop(0)


    if row['awayscore'] > row['homescore']:
        teamDict[awayteam]['wins'].append(1)
        teamDict[hometeam]['wins'].append(0)
        
    else:
        teamDict[hometeam]['wins'].append(1)
        teamDict[awayteam]['wins'].append(0)


    teamDict[awayteam]['spread'].append(row['awayscore'] - row['homescore'])
    teamDict[hometeam]['spread'].append(row['homescore'] - row['awayscore'])

    if row['homescore'] - row['awayscore'] > -1 * row['homespreadopen']:
        teamDict[hometeam]['returns'].append(1)
        teamDict[awayteam]['returns'].append(0)
    elif row['homescore'] - row['awayscore'] == -1 * row['homespreadopen']: 
        teamDict[hometeam]['returns'].append(.5)
        teamDict[awayteam]['returns'].append(.5)
    else:
        teamDict[awayteam]['returns'].append(1)
        teamDict[hometeam]['returns'].append(0)

    teamDict[awayteam]['games'] += 1
    teamDict[hometeam]['games'] += 1

In [8]:
df['AwayWins'] = (df['AwayWins'] - df['AwayWins'].mean()) / df['AwayWins'].std()
df['AwaySpread'] = (df['AwaySpread'] - df['AwaySpread'].mean()) / df['AwaySpread'].std()
df['AwayReturns'] = (df['AwayReturns'] - df['AwayReturns'].mean()) / df['AwayReturns'].std()
df['HomeWins'] = (df['HomeWins'] - df['HomeWins'].mean()) / df['HomeWins'].std()
df['HomeSpread'] = (df['HomeSpread'] - df['HomeSpread'].mean()) / df['HomeSpread'].std()
df['HomeReturns'] = (df['HomeReturns'] - df['HomeReturns'].mean()) / df['HomeReturns'].std()

In [9]:
df['AwayChar'] = df['AwayWins'] + df['AwaySpread'] + df['AwayReturns']
df['HomeChar'] = df['HomeWins'] + df['HomeSpread'] + df['HomeReturns']

In [ ]:
chardf = df.copy(deep=True)
chardf[(chardf.HomeGames < N) | (chardf.AwayGames < N)].index
chardf.drop(chardf[(chardf.HomeGames < N) | (chardf.AwayGames < N)].index, inplace= True)

In [11]:
chardf['Momentum'] = np.where(chardf['homespreadopen'] < 0, (chardf['HomeChar'] - chardf['AwayChar']), (chardf['AwayChar'] - chardf['HomeChar']))
chardf['games'] = chardf.groupby('date')['date'].transform('size')
chardf['ranks'] = chardf.groupby('date')['Momentum'].rank()

chardf['thresh'] = chardf.ranks / chardf.games
chardf['buy'] = chardf['thresh'].ge(.8)
chardf['sell'] = chardf['thresh'].le(.2)
#chardf.to_stata('MLBSpreadChar.dta')

In [12]:
chardf['overMom'] = chardf['HomeChar'] + chardf['AwayChar']
chardf['overRanks'] = chardf.groupby('date')['overMom'].rank()

chardf['overthresh'] = chardf.overRanks / chardf.games
chardf['overBuy'] = chardf['overthresh'].ge(.8)
chardf['overSell'] = chardf['overthresh'].le(.2)

In [13]:

chardf['weight'] = chardf['buy'] / chardf['games'] - chardf['sell'] / chardf['games']
chardf['smallocreturn'] = chardf['weight'] * chardf['spreadocreturn']
chardf['smallclosereturn'] = chardf['weight'] * chardf['spreadclosereturn']

chardf['overWeight'] = chardf['overBuy'] / chardf['games'] - chardf['overSell'] / chardf['games']
chardf['smalloverocreturn'] = chardf['overWeight'] * chardf['overocreturn']
chardf['smalloverclosereturn'] = chardf['overWeight'] * chardf['overclosereturn']

chardf.to_stata('NBASpreadChar.dta')

In [14]:
weights  = chardf[chardf.weight != 0]

NBAmomspreadret = pd.DataFrame(weights.groupby('date')['smallocreturn'].sum())
NBAmomspreadret.rename(columns = {'smallocreturn':'spreadocreturn'}, inplace=True)
NBAmomspreadret['spreadclosereturns'] = weights.groupby('date')['smallclosereturn'].sum()
NBAmomspreadret['spreadopenreturn'] = NBAmomspreadret['spreadocreturn'] + NBAmomspreadret['spreadclosereturns']
NBAmomspreadret['spreadtrading'] = NBAmomspreadret['spreadocreturn'] - NBAmomspreadret['spreadclosereturns']

means = NBAmomspreadret.mean()*GAMES * 2/5
std = NBAmomspreadret.std() * np.sqrt(GAMES) * np.sqrt(2/5)
print(means)
print(means / std)

returns = ['spreadocreturn', 'spreadclosereturns', 'spreadopenreturn', 'spreadtrading']
tstats = {}
for x in returns:
    tstats[x] = sm.ols(formula="{}~1".format(x), data = NBAmomspreadret).fit().tvalues['Intercept']
print(tstats)

spreadocreturn        6.671246
spreadclosereturns    1.178428
spreadopenreturn      7.849674
spreadtrading         5.492818
dtype: float64
spreadocreturn        1.261974
spreadclosereturns    0.123123
spreadopenreturn      0.819222
spreadtrading         0.452590
dtype: float64
{'spreadocreturn': 3.254924659760973, 'spreadclosereturns': 0.3175632851893764, 'spreadopenreturn': 2.112964078351039, 'spreadtrading': 1.1673356047178332}


In [26]:
overweights  = chardf[chardf.overWeight != 0]
NBAmomoverret = pd.DataFrame(overweights.groupby('date')['smalloverocreturn'].sum())
NBAmomoverret.rename(columns = {'smalloverocreturn':'overocreturn'}, inplace=True)
NBAmomoverret['overclosereturns'] = overweights.groupby('date')['smalloverclosereturn'].sum() 
NBAmomoverret['overopenreturn'] = NBAmomoverret['overocreturn'] + NBAmomoverret['overclosereturns']
NBAmomoverret['overtrading'] = NBAmomoverret['overocreturn'] - NBAmomoverret['overclosereturns']

means = NBAmomoverret.mean()*GAMES * 2/5
std = NBAmomoverret.std() * np.sqrt(GAMES) * np.sqrt(2/5)
print(means)
print(means / std)

returns = ['overocreturn', 'overclosereturns', 'overopenreturn', 'overtrading']
tstats = {}
for x in returns:
    tstats[x] = sm.ols(formula="{}~1".format(x), data = NBAmomoverret).fit().tvalues['Intercept']
print(tstats)

#NBAmomret.to_stata('NBATrading.dta')

overocreturn       -1.889427
overclosereturns   -3.813448
overopenreturn     -5.702875
overtrading         1.924022
dtype: float64
overocreturn       -0.513464
overclosereturns   -0.393820
overopenreturn     -0.589044
overtrading         0.174999
dtype: float64
{'overocreturn': -1.3243426375767702, 'overclosereturns': -1.0157542642500574, 'overopenreturn': -1.5192811631496295, 'overtrading': 0.4513642701030753}
